In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import seaborn as sns
from pandas import ExcelWriter
from pandas import ExcelFile
import sample_utils

%matplotlib inline

In [26]:
file_path = '2020_LER_20201102_testset_V04.xlsx'

gen_seed_list = [9226,  807,  524, 2646, 1638, 3741,  328, 7822, 9461, 8437]
sample_num = 750
result_total = []

for i in range(10):
    
    gen_seed = gen_seed_list[i]
    
    # True Data    
    
    file_path = '2020_LER_20201102_testset_V04.xlsx'

    data = np.load('../data_handler/'+file_path+'.npy', allow_pickle=True)
    num_in_cycle = [250]*10
    num_of_cycle = 10

    X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]
    test_real = Y_all

    # for pair plot indexing
    test_X_cycle = X_per_cycle

    y_mean = np.mean(Y_all, axis=0, dtype=np.float32)
    y_std = np.std(Y_all, axis=0, dtype=np.float32)

    # Gen Data

    result = 'test_specificnaive_date_210130_data_2020_LER_20201008_V008.xlsx_model_mlp_gan1_seed_{}_lr_5e-05_0.0001_0.0005_hidden_dim_100_50_batch_size_32_noise_d_100_sample_num_{}_tr_num_in_cycle_50_pdrop_0.9.npy'.format(gen_seed, sample_num)
    print(result)
    test_gen = np.load(result, allow_pickle=True)

    num_of_cycle = 10

    test_gen_sample_num = int(test_gen.shape[0]/num_of_cycle)
    gen_num_in_cycle = [test_gen_sample_num]*num_of_cycle
    print(test_gen_sample_num)
    print(gen_num_in_cycle)
    
    
    min1_per_out_v = test_real.min(axis=0)
    max1_per_out_v = test_gen.max(axis=0)

    min2_per_out_v = test_real.min(axis=0)
    max2_per_out_v = test_real.max(axis=0)
    print("minimum y value for each varaible:", min2_per_out_v)
    print("maximum y value for each varaible:", max2_per_out_v)

    min_per_out_v = np.minimum(min1_per_out_v, min2_per_out_v)
    max_per_out_v = np.maximum(max1_per_out_v, max2_per_out_v)

    # EMD  

    EMD_1d_score_list, EMD_2d_score_list = sample_utils.EMD_all_pair_each_X_integral(generated_samples = test_gen, real_samples = test_real, num_coordinate=10, num_of_cycle=num_of_cycle, num_in_gen_list=gen_num_in_cycle, num_in_real_list = num_in_cycle, min_list = min_per_out_v, max_list = max_per_out_v) 

    print(EMD_1d_score_list.shape)
    print(EMD_2d_score_list.shape)

    EMD_score_cat = np.hstack((EMD_1d_score_list, EMD_2d_score_list))

    EMD_score = np.mean(EMD_score_cat)
    EMD_score_list_integral = np.mean(EMD_score_cat,axis=1)
    EMD_std = np.sqrt(np.mean((EMD_score - EMD_score_list_integral)**2))
    
    
    result_total.append(EMD_score_list_integral)
    
    print(result_total)
    
#     for factor in [1, 6]:

#         print("factor", factor)

#         ##############################################################################################################
#         """
#         for i in range(6):
#             print(np.min(total_result[:,i]))
#             print(np.max(total_result[:,i]))
#             print(np.min(real[:,i]))
#             print(np.max(real[:,i]))
#         """

#         # generated sample
#         print(sum(num_in_cycle[:factor]),sum(num_in_cycle[:factor])+num_in_cycle[factor])
#         test_gen_matrix_plot = test_gen[sum(gen_num_in_cycle[:factor]):sum(gen_num_in_cycle[:factor])+gen_num_in_cycle[factor]]
#         print("real sample shape:", test_gen_matrix_plot.shape)

#         # real sample
#         test_real_matrix_plot = test_real[sum(num_in_cycle[:factor]):sum(num_in_cycle[:factor])+num_in_cycle[factor]]
#         print("test sample shape:", test_real_matrix_plot.shape)
#         print()

#         # Assigning class(real: 0, gen:1 ) for visualization

#         print("Assigning class for visualization (real: 0, gen:1 )")

#         test_gen_cls = np.ones((gen_num_in_cycle[factor], 1))
#         print("real A sample class shape:", test_gen_cls.shape)
#         test_real_cls = np.zeros((num_in_cycle[factor], 1))
#         print("real B sample class shape:", test_real_cls.shape)

#         test_temp1 = np.hstack([test_gen_matrix_plot, test_gen_cls])
#         test_temp2 = np.hstack([test_real_matrix_plot, test_real_cls])

#         # TOTAL PANDAS DATA
#         total_test_matrix_plot = np.vstack([test_temp1, test_temp2])
#         # print(total_matrix_plot.shape)

#         # A: Ioff, B: IDSat, C: IDlin, D: VTsat, E:VTlin, F:SS, G: classes
#         df1_test = pd.DataFrame(total_test_matrix_plot, columns=['Ioff', 'IDSat',  'IDlin',  'VTsat', 'VTlin', 'SS', 'classes'])
#         # print(df1)

#     #     print(np.min(test_min_list))
#     #     print(np.max(test_max_list, axis=1))
#         X_test = X_per_cycle[factor]

#         print("X input", X_test)

#         g = sns.pairplot(df1_test, hue = 'classes')
#         for i in range(6):
#             g.fig.suptitle("CGAN Amp: {:.4f}, Corr x: {:.4f}, Corr y: {:.4f}, EMD integral: {:.3f}".format(X_test[0], X_test[1], X_test[2], EMD_score_list_integral[factor]), y=1.04, fontsize=30)
#             g.axes[i,i].set_xlim((min_per_out_v[i], max_per_out_v[i]))
#             g.axes[i,i].set_ylim((min_per_out_v[i], max_per_out_v[i]))

#         for ax in g.axes.flatten():
#             ax.ticklabel_format(style='sci', scilimits=(0,0), axis='both')

#         plt.rcParams.update({'font.size': 15})
#         plt.tight_layout()
#         plt.show()
#             ### 파란색이 real 주황색이 generated

test_specificnaive_date_210130_data_2020_LER_20201008_V008.xlsx_model_mlp_gan1_seed_9226_lr_5e-05_0.0001_0.0005_hidden_dim_100_50_batch_size_32_noise_d_100_sample_num_750_tr_num_in_cycle_50_pdrop_0.9.npy
750
[750, 750, 750, 750, 750, 750, 750, 750, 750, 750]
minimum y value for each varaible: [3.0220e-13 2.6090e-05 1.1500e-05 3.1800e-01 3.3000e-01 6.0765e+01]
maximum y value for each varaible: [8.3370e-12 4.5900e-05 1.8750e-05 3.9400e-01 3.9700e-01 6.3893e+01]
(10, 6)
(10, 15)
[array([0.22537137, 0.16876779, 0.15461261, 0.19590949, 0.1834912 ,
       0.16712808, 0.08119492, 0.16131858, 0.12196918, 0.07925279])]
test_specificnaive_date_210130_data_2020_LER_20201008_V008.xlsx_model_mlp_gan1_seed_807_lr_5e-05_0.0001_0.0005_hidden_dim_100_50_batch_size_32_noise_d_100_sample_num_750_tr_num_in_cycle_50_pdrop_0.9.npy
750
[750, 750, 750, 750, 750, 750, 750, 750, 750, 750]
minimum y value for each varaible: [3.0220e-13 2.6090e-05 1.1500e-05 3.1800e-01 3.3000e-01 6.0765e+01]
maximum y value for 

(10, 6)
(10, 15)
[array([0.22537137, 0.16876779, 0.15461261, 0.19590949, 0.1834912 ,
       0.16712808, 0.08119492, 0.16131858, 0.12196918, 0.07925279]), array([0.24500518, 0.18276779, 0.16448133, 0.21414764, 0.18072835,
       0.16268385, 0.07087939, 0.14673922, 0.10917517, 0.07790477]), array([0.1807975 , 0.13896742, 0.20938869, 0.18734117, 0.22814807,
       0.16386316, 0.06816458, 0.1783648 , 0.11792385, 0.07879372]), array([0.26909597, 0.1515674 , 0.2010023 , 0.19205485, 0.22002226,
       0.14474717, 0.05891352, 0.20521617, 0.14150549, 0.06477219]), array([0.18554901, 0.17706958, 0.2168837 , 0.22372239, 0.19060351,
       0.16027514, 0.06253488, 0.19481099, 0.09920894, 0.09834102]), array([0.24018833, 0.19105287, 0.25921701, 0.20987159, 0.17415065,
       0.14916386, 0.08394676, 0.13633975, 0.12609963, 0.07587061]), array([0.21238508, 0.16485071, 0.19675266, 0.16936714, 0.20491497,
       0.15885956, 0.05309818, 0.13334887, 0.14478099, 0.08300172]), array([0.24260384, 0.14605805,

In [24]:
np.mean(result_total, axis=0)

array([0.23019361, 0.16472518, 0.2050434 , 0.20150969, 0.20367673,
       0.17017566, 0.06600316, 0.16912136, 0.11813598, 0.08065658])

In [25]:
np.std(result_total, axis=0)

array([0.02736001, 0.03105772, 0.03214527, 0.01892219, 0.02780586,
       0.03895784, 0.00937325, 0.0265723 , 0.01462277, 0.00880808])